In [7]:
#**********            STEP  1          ************

import os, shutil, csv

#creo il file csv completo di intestazione se già non esiste, e aggiorno il file csv
def write_csv(name, typ, size):
    if 'recap.csv' not in os.listdir():
        with open('recap.csv', 'w', newline='') as recap:
            writer = csv.writer(recap)        
            writer.writerow(['name', 'type', 'size(B)'])
    with open('recap.csv', 'a', newline='') as recap:
        writer = csv.writer(recap)        
        writer.writerow([name, typ, size])

#creo la funzione che mi permette di spostare i file nella cartella assegnata, poi stampo le informazioni e aggiorno il recap
#uso il try/except nel caso in cui esistesse già un file uguale nella cartella di destinazione.
def move_write(file, typ, path):
        try:    
            name = os.path.splitext(file)[0]
            size = os.stat(file).st_size
            shutil.move(file, os.path.join(os.getcwd(), path))
            write_csv(name, typ, size)
            print('{} type: {} size: {}B'.format(name, typ, size))
        except:
            print('Impossibile spostare il file {}. Esiste già nella cartella di destinazione.'.format(file))
        
#creo la funzione principale che mi permette di verificare l'estensione del file. ho messo le estensioni più conosciute,
#nel caso in cui il file fosse di un'estensione sconosciuta stampo un messaggio di errore e proseguo con il ciclo,
#bypassando le cartelle e il file csv. inoltre, nel caso non esistessero già, creo le sottocartelle di files
def main_func(file):
        if file.endswith('.jpeg') or file.endswith('.jpg') or file.endswith('.png') or file.endswith('.gif'):
            if not os.path.exists('images'):
                os.makedirs('images')
            move_write(file, 'image', 'images')

        elif file.endswith('.txt') or file.endswith('.odt') or file.endswith('.doc') or file.endswith('.docx'):
            if not os.path.exists('docs'):
                os.makedirs('docs')
            move_write(file, 'text', 'docs')

        elif file.endswith('.mp3') or file.endswith('.wav') or file.endswith('.wma') or file.endswith('.flac'):
            if not os.path.exists('audio'):
                os.makedirs('audio') 
            move_write(file, 'audio', 'audio')
        
        elif file == 'images' or file == 'docs' or file == 'audio' or file == 'recap.csv':
            pass
        
        else:
            print("Il file {} ha un'estensione sconosciuta. La sua posizione non verrà modificata.".format(file))

#ho pensato che questo file originariamente si trovi nella cartella principale FileOrganizer, 
#quindi con questa parte dello script mi sposto nella cartella file dalle due possibili cartelle in cui mi potrei trovare
# mi sembra un po' macchinoso ma non ho trovato alternative, ho cercato di utilizzare un codice universale che vada bene su qualsiasi computer
if os.getcwd().endswith('images'):
    os.chdir('..')
elif os.getcwd().endswith('FileOrganizer'):
    os.chdir('./files')

    
#richiamo la funzione principale per ogni file nella cartella files, se non è vuota
if os.listdir():
    for file in sorted(os.listdir()):
        main_func(file)

else:
    print('La cartella "files" è vuota.')
    


1 type: image size: 1801934B
2 type: image size: 1849975B
Crystalize type: audio size: 11985645B
bw type: image size: 94926B
ciao type: text size: 12B
daffodil type: image size: 24657B
eclipse type: image size: 64243B
pippo type: text size: 8299B
song1 type: audio size: 1087849B
song2 type: audio size: 764176B
trump type: image size: 10195B


In [8]:
#**********            STEP  3          ************

from PIL import Image, UnidentifiedImageError
import numpy as np, tabulate, os

#creo una funzione che, partendo da un file immagine, lo converte in un array    
def file_to_tab(file, lista):
    
    try:
        A = np.array(Image.open(file))
  
        #verifico se il file è BW, RGB o RGBA e ne estraggo i dati mettendoli in una tupla all'interno di una lista
        if np.ndim(A) == 2:
            lista += [(os.path.splitext(file)[0], 'BW', np.shape(A)[0], np.shape(A)[1],  np.mean(A), 0, 0, 0, 0)]

        elif np.ndim(A) == 3 and np.shape(A)[2] == 3:
            lista += [(os.path.splitext(file)[0], 'RGB', np.shape(A)[0], np.shape(A)[1], 0, np.mean(A[:, :, 0]), np.mean(A[:, :, 1]), np.mean(A[:, :, 2]), 0)]

        elif np.ndim(A) == 3 and np.shape(A)[2] == 4:
            lista += [(os.path.splitext(file)[0], 'RGBA', np.shape(A)[0], np.shape(A)[1], 0, np.mean(A[:, :, 0]), np.mean(A[:, :, 1]), np.mean(A[:, :, 2]), np.mean(A[:, :, 3]))]

        else:
            print("Non riesco a decifrare l'immagine che mi hai passato, ci sono troppi livelli.")
    
    except UnidentifiedImageError:
        print("Il file {} non è un'immagine".format(file))
        
    except Exception as e:
        print(str(e))
        
    #ritorno la lista che mi servirà per creare la tabella
    return lista


#ho pensato questo file originariamente per funzionare nella cartella principale FileOrganizer, 
#quindi con questa parte dello script mi sposto nelle cartelle file e poi images. 
#mi sembra un po' macchinoso ma non ho trovato alternative, ho cercato di utilizzare un codice universale che vada bene su qualsiasi computer
try:
    if os.getcwd().endswith('FileOrganizer'):
        os.chdir('./files')

    if os.getcwd().endswith('files'):
        os.chdir('./images')

    #creo la lista da riempire con i valori delle immagini, e l'intestazione della tabella
    lista = []
    hd = ['name', 'type', 'height', 'width', 'grayscale', 'R', 'G', 'B', 'ALPHA']

    #richiamo la funzione file_to_tab per ogni file nella cartella images
    for file in sorted(os.listdir()):
        file_to_tab(file, lista)

    #stampo la tabella con i dati aggiornati, se la cartella images non è vuota
    if lista:
        print(tabulate.tabulate(lista, hd, tablefmt='fancy_grid', floatfmt='.2f'))

    else:
        print('La cartella "images" è vuota!')

except:
    print('Non trovo la cartella "images". Esegui prima lo script 1 per crearla e riprova.')

╒══════════╤════════╤══════════╤═════════╤═════════════╤════════╤════════╤════════╤═════════╕
│ name     │ type   │   height │   width │   grayscale │      R │      G │      B │   ALPHA │
╞══════════╪════════╪══════════╪═════════╪═════════════╪════════╪════════╪════════╪═════════╡
│ 1        │ RGB    │     3000 │    4000 │        0.00 │ 118.14 │ 116.85 │ 122.17 │    0.00 │
├──────────┼────────┼──────────┼─────────┼─────────────┼────────┼────────┼────────┼─────────┤
│ 2        │ RGB    │     4000 │    3000 │        0.00 │ 126.47 │ 124.39 │ 127.06 │    0.00 │
├──────────┼────────┼──────────┼─────────┼─────────────┼────────┼────────┼────────┼─────────┤
│ bw       │ BW     │      512 │     512 │       21.48 │   0.00 │   0.00 │   0.00 │    0.00 │
├──────────┼────────┼──────────┼─────────┼─────────────┼────────┼────────┼────────┼─────────┤
│ daffodil │ RGB    │      500 │     335 │        0.00 │ 109.25 │  85.56 │   4.97 │    0.00 │
├──────────┼────────┼──────────┼─────────┼─────────────┼────